In [8]:
from PIL import Image, ImageOps

In [9]:

# Load the base image
base_image_path = 'base-image.png'
base_image = Image.open(base_image_path).convert("RGBA")


In [10]:

# Define the final image size and the position to center the base image
final_size = (800, 800)
base_position = ((final_size[0] - base_image.width) // 2, (final_size[1] - base_image.height) // 2)



In [39]:
import numpy as np

def generate_distinct_hsl_colors(total_colors):
    """Generate a list of distinct HSL colors by evenly distributing hues and adjusting lightness and saturation for variety."""
    hues = np.linspace(0, 360, total_colors, endpoint=False)
    colors = []
    for i, hue in enumerate(hues):
        # Alternate lightness and saturation to increase distinction between colors
        lightness = 25 + (i % 4) * 15  # Cycle through 25%, 40%, 55%, 70%
        saturation = 50 if (i // 4) % 2 == 0 else 100  # Alternate saturation between 50% and 100%
        colors.append(f'hsl({int(hue)}, {saturation}%, {lightness}%)')
    return colors

def generate_complementary_hsl(color):
    """Generate a complementary HSL color."""
    hue, saturation, lightness = [int(part.split('%')[0]) for part in color[4:-1].split(',')]
    comp_hue = (hue + 180) % 360
    # Invert lightness and adjust saturation for the complementary color
    comp_lightness = 100 - lightness
    comp_saturation = 100 if saturation == 50 else 50
    return f'hsl({comp_hue}, {comp_saturation}%, {comp_lightness}%)'

def create_color_pairs(colors):
    """Pair each color with its complementary color."""
    return [(color, generate_complementary_hsl(color)) for color in colors]

# Generate total distinct colors for the entire set and then divide by categories
total_required_colors = 9999
all_colors = generate_distinct_hsl_colors(total_required_colors)

# Split the colors into four categories
split_index = total_required_colors // 4
dark_colors = all_colors[:split_index]
light_colors = all_colors[split_index:2*split_index]
vibrant_colors = all_colors[2*split_index:3*split_index]
classic_muted_colors = all_colors[3*split_index:]

# Generate color pairs for each category
dark_pairs = create_color_pairs(dark_colors)
light_pairs = create_color_pairs(light_colors)
vibrant_pairs = create_color_pairs(vibrant_colors)
classic_pairs = create_color_pairs(classic_muted_colors)

# Combine all pairs into the final list ensuring each pair is visually distinct
colors = dark_pairs + light_pairs + vibrant_pairs + classic_pairs

len(colors)


9999

In [40]:



for i, (background_color, fill_color) in enumerate(colors):
    # Create a new blank image with the final size and background color
    final_image = Image.new('RGBA', final_size, background_color)
    
    # Extract the alpha channel from the base image for smooth edges
    alpha = base_image.split()[-1]

    # Create an image for recoloring that matches the base image size
    recolor_image = Image.new('RGB', base_image.size, fill_color)
    recolor_image.putalpha(alpha)  # Apply the original alpha channel

    # Ensure smooth edges by using the alpha channel effectively
    final_image.paste(recolor_image, base_position, recolor_image)
    
    # Save the final image
    final_image_path = f'{i+1}.png'
    final_image.save(final_image_path)

print("All variations have been created and saved.")


All variations have been created and saved.


In [50]:
import json
import re
import colorsys

def parse_hsl(hsl_str):
    """Parse HSL values from an HSL string."""
    hsl_match = re.match(r'hsl\((\d+), (\d+)%, (\d+)%\)', hsl_str)
    if hsl_match:
        return tuple(map(int, hsl_match.groups()))
    else:
        raise ValueError(f"Could not parse HSL values from '{hsl_str}'")

def hsl_to_hex(h, s, l):
    """Convert HSL color to HEX format."""
    rgb = colorsys.hls_to_rgb(h / 360, l / 100, s / 100)
    return '#%02x%02x%02x' % tuple(int(c * 255) for c in rgb)

def prepare_data(colors):
    """Prepare data for JSON output, converting colors from HSL to HEX."""
    data = []
    for index, (bg_hsl, fg_hsl) in enumerate(colors):
        try:
            bg_h, bg_s, bg_l = parse_hsl(bg_hsl)
            fg_h, fg_s, fg_l = parse_hsl(fg_hsl)
            bg_hex = hsl_to_hex(bg_h, bg_s, bg_l)
            fg_hex = hsl_to_hex(fg_h, fg_s, fg_l)
            data.append({
                "name": str((index)+1),
                "description": f"Monk Buidler #{index+1}",
                "background": bg_hex,
                "foreground": fg_hex
            })
        except ValueError as e:
            print(f"Error processing color at index {index}: {e}")
            continue  # Skip this pair and move on to the next
    return data

# Assuming 'colors' is already defined and filled with HSL color pairs
data_to_save = prepare_data(colors[:9420])

# Write to JSON file
with open('metadata.json', 'w') as file:
    json.dump(data_to_save, file, indent=4)
